**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
from keras.optimizers import sgd, Adam
from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten,Dropout

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The reason why epsilon is essential is that because it enables the agent to look for other actions that could be better or not, in other states. It acts as a decision boundary to tell if this further exploration is necessary or if the existing action (learned) is sufficient.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=50 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

Positions arrays are used indicate the previous position of the rat on the board. As for the board ones, they posses information on the rewards for all the positions. 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, 4, size = 1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 14.5/22.0. Average score (-7.5)
Win/lose count 9.5/11.0. Average score (-4.5)
Win/lose count 4.5/6.0. Average score (-3.5)
Win/lose count 11.5/10.0. Average score (-2.25)
Win/lose count 8.0/10.0. Average score (-2.2)
Win/lose count 10.5/8.0. Average score (-1.4166666666666667)
Win/lose count 11.0/14.0. Average score (-1.6428571428571428)
Win/lose count 10.0/15.0. Average score (-2.0625)
Win/lose count 15.0/13.0. Average score (-1.6111111111111112)
Win/lose count 6.0/14.0. Average score (-2.25)
Win/lose count 9.0/11.0. Average score (-2.227272727272727)
Win/lose count 13.5/17.0. Average score (-2.3333333333333335)
Win/lose count 8.0/14.0. Average score (-2.6153846153846154)
Win/lose count 4.5/13.0. Average score (-3.0357142857142856)
Win/lose count 6.5/9.0. Average score (-3.0)
Win/lose count 5.0/12.0. Average score (-3.25)
Win/lose count 9.0/12.0. Average score (-3.235294117647059)
Win/lose count 6.0/18.0. Average score (-3.7222222222222223)
Win/lose count 12.0/14.0. Ave

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




$$
\begin{align*} 
Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
          =E_{p^{\pi}}[r(s_0,a_0) + \gamma\sum_{t=1}^{\infty}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
          =r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a]\\
          =r(s,a) + \gamma E_{(s',a')\sim p(.|s,a)}\left(E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']\right)\\
          =E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']\right)\\
          =E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma Q^{\pi}(s',a') \right)
\end{align*}
$$

$$
\begin{align*} 
Q^*(s,a)
Q^*(s,a) = \underset{(a',\pi')}{\max}\left[E_{(s',a')\sim p(.|s,a)}\left(r(s,a)+\gamma Q^{(a',\pi')}(s',a')\right)\right]\\
 = \underset{(a',\pi')}{\max}\left[r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )Q^{(a',\pi')}(s',a')\right]\\
= r(s,a) + \gamma * \underset{(a',\pi')}{\max}\left[ \sum(p(s_{t+1}=s' |s_t = s, a_t = a )Q^{(a',\pi')}(s',a')\right]\\ 
\end{align*}
$$

We have seen previously that $Q^*$ is the solution of $F(Q) = 0$ with $$F(Q) = E_{s' \sim \pi^*(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q(s',a') - Q(s,a)\right)$$

Hence, minimizing $\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r(s,a) +\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$ will give the optimal policy.


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            del self.memory[0]

    def random_access(self):
        return self.memory[np.random.choice(len(self.memory))]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape((1,5,5,self.n_state))))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            st, nx_st, act, rw, g_o = self.memory.random_access()
            input_states[i] = st
            target_q[i] = self.model.predict([[st]])
            
            if game_over_:
                target_q[i][act] = rw
           
            else:
                p = self.model.predict([[nx_st]])
                mx_q = np.max(p)
                target_q[i][act] = rw + self.discount*mx_q
      
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights = 'model.h5',name_model = 'model.json'):
        self.model.save_weights(name_weights, overwrite = True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights = 'model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr = 0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Flatten(input_shape = (5,5,self.n_state)))
        model.add(Dense(24, activation = 'relu'))
        model.add(Dense(24, activation = 'relu'))
        model.add(Dense(4, activation = 'linear'))
        model.compile(loss = 'mse', optimizer = Adam(lr = lr))
        
        model.compile(sgd(lr = lr, decay = 1e-4, momentum = 0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/100 | Loss 0.0472 | Win/lose count 4.0/5.0 (-1.0)
Epoch 001/100 | Loss 0.0769 | Win/lose count 11.5/12.0 (-0.5)
Epoch 002/100 | Loss 0.1852 | Win/lose count 3.0/1.0 (2.0)
Epoch 003/100 | Loss 0.3900 | Win/lose count 6.0/5.0 (1.0)
Epoch 004/100 | Loss 0.8750 | Win/lose count 4.5/8.0 (-3.5)
Epoch 005/100 | Loss 1.3274 | Win/lose count 3.5/7.0 (-3.5)
Epoch 006/100 | Loss 1.3702 | Win/lose count 5.0/2.0 (3.0)
Epoch 007/100 | Loss 1.5081 | Win/lose count 3.5/4.0 (-0.5)
Epoch 008/100 | Loss 1.7715 | Win/lose count 6.5/5.0 (1.5)
Epoch 009/100 | Loss 1.5550 | Win/lose count 2.0/7.0 (-5.0)
Epoch 010/100 | Loss 1.3798 | Win/lose count 2.0/2.0 (0.0)
Epoch 011/100 | Loss 1.4926 | Win/lose count 2.5/4.0 (-1.5)
Epoch 012/100 | Loss 1.3727 | Win/lose count 4.0/5.0 (-1.0)
Epoch 013/100 | Loss 1.3894 | Win/lose count 8.0/1.0 (7.0)
Epoch 014/100 | Loss 1.3978 | Win/lose count 2.0/5.0 (-3.0)
Epoch 015/100 | Loss 1.4585 | Win/lose count 5.0/0 (5.0)
Epoch 016/100 | Loss 1.5914 | Win/lose count 8.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [15]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
         
        model = Sequential()
        model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu', input_shape = (5,5,self.n_state)))
        model.add(Dropout(rate = 0.1))
        model.add(Conv2D(32, (3, 3), activation = 'relu'))
        model.add(Flatten())
        model.add(Dense(32))
        model.add(Dropout(rate = 0.1))
        model.add(Dense(4))
        
        model.compile(sgd(lr = lr, decay = 1e-4, momentum = 0.0), "mse")
        self.model = model

In [16]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/100 | Loss 0.0241 | Win/lose count 3.0/3.0 (0.0)
Epoch 001/100 | Loss 0.0113 | Win/lose count 1.0/6.0 (-5.0)
Epoch 002/100 | Loss 0.0249 | Win/lose count 8.0/4.0 (4.0)
Epoch 003/100 | Loss 0.0341 | Win/lose count 3.0/4.0 (-1.0)
Epoch 004/100 | Loss 0.0587 | Win/lose count 2.5/4.0 (-1.5)
Epoch 005/100 | Loss 0.0597 | Win/lose count 6.0/1.0 (5.0)
Epoch 006/100 | Loss 0.1169 | Win/lose count 3.5/3.0 (0.5)
Epoch 007/100 | Loss 0.1273 | Win/lose count 2.5/2.0 (0.5)
Epoch 008/100 | Loss 0.1620 | Win/lose count 1.0/2.0 (-1.0)
Epoch 009/100 | Loss 0.1478 | Win/lose count 5.0/3.0 (2.0)
Epoch 010/100 | Loss 0.1408 | Win/lose count 4.5/0 (4.5)
Epoch 011/100 | Loss 0.1705 | Win/lose count 3.0/1.0 (2.0)
Epoch 012/100 | Loss 0.1581 | Win/lose count 6.5/3.0 (3.5)
Epoch 013/100 | Loss 0.1491 | Win/lose count 3.0/0 (3.0)
Epoch 014/100 | Loss 0.1313 | Win/lose count 4.5/0 (4.5)
Epoch 015/100 | Loss 0.1118 | Win/lose count 3.5/1.0 (2.5)
Epoch 016/100 | Loss 0.1098 | Win/lose count 1.5/0 (1.5)
E

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [17]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 5.5/2.0. Average score (3.5)
Win/lose count 4.5/1.0. Average score (3.5)
Win/lose count 6.5/1.0. Average score (4.166666666666667)
Win/lose count 6.5/2.0. Average score (4.25)
Win/lose count 6.5/3.0. Average score (4.1)
Win/lose count 1.5/0. Average score (3.6666666666666665)
Win/lose count 9.0/2.0. Average score (4.142857142857143)
Win/lose count 12.5/5.0. Average score (4.5625)
Win/lose count 9.5/4.0. Average score (4.666666666666667)
Win/lose count 2.5/2.0. Average score (4.25)
Win/lose count 9.5/2.0. Average score (4.545454545454546)
Win/lose count 6.0/4.0. Average score (4.333333333333333)
Win/lose count 7.0/0. Average score (4.538461538461538)
Win/lose count 7.5/1.0. Average score (4.678571428571429)
Win/lose count 5.5/3.0. Average score (4.533333333333333)
Win/lose count 3.5/1.0. Average score (4.40625)
Win/lose count 6.0/4.0. Average score (4.264705882352941)
Win/lose count 2.0/1.0. Average score (4.083333333333333)
Win/lose count 15.0/3.0. Averag

We can observe that the performances of the CNN are better than those of FC. We can also say that the agent is not exploring the map as it should (not exploring it all). 

In [18]:
env = Environment(grid_size=size, max_time=T,temperature=0.8)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 18.5/2.0. Average score (16.5)
Win/lose count 17.0/2.0. Average score (15.75)
Win/lose count 23.0/1.0. Average score (17.833333333333332)
Win/lose count 20.0/1.0. Average score (18.125)
Win/lose count 28.0/2.0. Average score (19.7)
Win/lose count 19.0/2.0. Average score (19.25)
Win/lose count 22.5/0. Average score (19.714285714285715)
Win/lose count 42.5/3.0. Average score (22.1875)
Win/lose count 19.0/2.0. Average score (21.61111111111111)
Win/lose count 19.0/3.0. Average score (21.05)
Win/lose count 15.0/0. Average score (20.5)
Win/lose count 25.0/3.0. Average score (20.625)
Win/lose count 19.5/0. Average score (20.53846153846154)
Win/lose count 41.0/1.0. Average score (21.928571428571427)
Win/lose count 28.0/3.0. Average score (22.133333333333333)
Win/lose count 8.5/0. Average score (21.28125)
Win/lose count 15.5/0. Average score (20.941176470588236)
Win/lose count 36.5/1.0. Average score (21.75)
Win/lose count 9.0/1.0. Average score (21.02631578947368

When increasing the temperature, we observe an improvement in the map exploration. 

In [19]:
HTML(display_videos('cnn_test10.mp4'))

In [20]:
HTML(display_videos('fc_test10.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [21]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    agent.epsilon = 1.

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        agent.epsilon = agent.epsilon * 0.95
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train =True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
   
    
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b
        
    def act(self, action, train= False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, :-2] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
            self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                        self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size,self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [22]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/100 | Loss 0.4176 | Win/lose count 14.0/24.100000000000065 (-10.100000000000065)
Epoch 001/100 | Loss 0.1455 | Win/lose count 7.5/28.400000000000077 (-20.900000000000077)
Epoch 002/100 | Loss 0.2125 | Win/lose count 6.5/24.300000000000068 (-17.800000000000068)
Epoch 003/100 | Loss 0.2430 | Win/lose count 11.0/28.700000000000102 (-17.700000000000102)
Epoch 004/100 | Loss 0.2418 | Win/lose count 10.0/29.000000000000068 (-19.000000000000068)
Epoch 005/100 | Loss 0.3409 | Win/lose count 3.0/23.900000000000055 (-20.900000000000055)
Epoch 006/100 | Loss 0.3075 | Win/lose count 7.0/20.70000000000003 (-13.700000000000031)
Epoch 007/100 | Loss 0.4363 | Win/lose count 12.5/20.90000000000003 (-8.40000000000003)
Epoch 008/100 | Loss 0.4730 | Win/lose count 12.5/23.800000000000043 (-11.300000000000043)
Epoch 009/100 | Loss 0.9091 | Win/lose count 11.0/21.60000000000002 (-10.60000000000002)
Epoch 010/100 | Loss 1.0479 | Win/lose count 17.5/23.100000000000065 (-5.600000000000065)
Epoch 011/

Epoch 092/100 | Loss 0.6063 | Win/lose count 14.0/17.999999999999993 (-3.999999999999993)
Epoch 093/100 | Loss 0.6020 | Win/lose count 9.5/16.399999999999963 (-6.899999999999963)
Epoch 094/100 | Loss 0.5705 | Win/lose count 18.0/13.799999999999967 (4.200000000000033)
Epoch 095/100 | Loss 0.5565 | Win/lose count 9.0/18.099999999999987 (-9.099999999999987)
Epoch 096/100 | Loss 0.4280 | Win/lose count 7.5/19.000000000000007 (-11.500000000000007)
Epoch 097/100 | Loss 0.3814 | Win/lose count 21.5/15.99999999999997 (5.50000000000003)
Epoch 098/100 | Loss 0.5044 | Win/lose count 11.5/17.29999999999998 (-5.799999999999979)
Epoch 099/100 | Loss 0.3474 | Win/lose count 14.5/17.199999999999974 (-2.6999999999999744)


In [ ]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***